# [모듈 2.0] 모델 빌딩하기 (No VPC 환경에서 실행하세요)

이 노트북은 아래 제공된 훈련 스크립트 (training_script 폴더)를 통하여 4가지의 훈련 방법을 소개 합니다. (**이 노트북의 소요시간은 약 20분 걸립니다.**)
- cifar10_keras_tf2.py
    - Keras를 사용하여 만든 훈련 스크립트 입니다.
- cifar10_keras_sm_tf2.py
    - 위의 Keras 훈련 스크립트를 SageMaker에서 훈련이 가능하게 변경한 스크립트 입니다.

위의 스크립트를 이용하여 아래와 같은 작업을 합니다.
- (1) cifar10_keras_tf2.py 를 이용하여 **SageMaker Notebook Instance 로컬 O/S**에서 실행하여 훈련 합니다.
- (2) cifar10_keras_sm_tf2.py 를 이용하여 **SageMaker Notebook Instance 로컬 O/S**에서 실행하여 훈련 합니다.
    - 이는 SageMaker의 Script Mode (Docker 에서 실행)를 실제 실행하지 않고 시뮬레이션 합니다.
- (3) cifar10_keras_sm_tf2.py 를 이용하여 **SageMaker Notebook Instance 로컬 Docker**에서 실행하여 훈련 합니다.
    - SageMaker의 **local mode** 라고 불리웁니다.
    - Docker 에는 1. 내장 딥러닝 다커 컨테이너(예: tensorflow)를 사용하고, 2.사용자가 정의한 훈련 스크립트(에: cifar10_keras_sm_tf2.py) 를 사용하여 훈련 합니다.
    - 필요시 의존성 패키지(예: python-cv2)를 딥러닝 다커 컨테이너에 설치하여 사용합니다.
    - 위에서 내장 딥러닝 다커 컨테이너를 사용자 정의 컨테이너를 사용할 수도 있습니다.
- (4) cifar10_keras_sm_tf2.py 를 이용하여 **SageMaker Notebook Cloud Hosting Cluster** 에서 실행하여 훈련 합니다.     
    - SageMaker Notebook Cloud Hosting Cluster 에서 지정한 EC2 Instance(예: ml.p3.2xlarge)를 생성하고, 여기에 Docker를 로딩하여 훈련을 한 후에, EC2 Instance는 종료 시킵니다.
---
본 노트북(notebook)은 SageMaker 상에서 Keras Sequential model을 학습하는 방법을 단계별로 설명합니다. 본 노트북에서 사용한 모델은 간단한 deep CNN(Convolutional Neural Network) 모델 입니다.
- 참고로, 본 모델은 25 epoch 학습 후에 검증셋의 정확도(accuracy)가 약 75%이고 50 epoch 학습 후에 검증셋의 정확도가 약 79% 입니다.

---
본 노트북은 아래 워크샵의 내용을 참조 합니다.
- SageMaker Workshop: Tensorflow-Keras 모델을 Amazon SageMaker에서 학습하기
    - https://github.com/daekeun-ml/tensorflow-in-sagemaker-workshop
    - 아래 노트북이 본 노트북의 원본 노트북 입니다.
        - https://github.com/daekeun-ml/tensorflow-in-sagemaker-workshop/blob/master/0_Running_TensorFlow_In_SageMaker.ipynb

---


# The dataset 확인

[CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html)은 머신 러닝에서 가장 유명한 데이터셋 중 하나입니다.
이 데이터셋은 10개의 다른 클래스로 구성된(클래스당 6,000장) 60,000장의 32x32 픽셀 이미지들로 구성되어 있습니다.
아래 그림은 클래스당 10장의 이미지들을 랜덤으로 추출한 결과입니다. 

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

본 실습에서 여러분들은 deep CNN을 학습하여 영상 분류(image classification) 작업을 수행합니다. 

### Tensorflow 2.0 버전 이상 확인
이 노트북은 tensorflow 2.0 이상을 사용 합니다. 커널은 **"Conda_tensorflow2_p36"** 혹은 아래 pip로 설치해서 하시면 됩니다. 

In [13]:
# !pip install tensorflow==2.0.0

In [14]:
import tensorflow as tf
import numpy as np
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [15]:
tf.__version__

'2.1.2'

# (1) Run the training locally

본 스크립트는 모델 학습에 필요한 인자값(arguments)들을 사용합니다. 모델 학습에 필요한 인자값들은 아래와 같습니다.

1. `model_dir` - 로그와 체크 포인트를 저장하는 경로
2. `train, validation, eval` - TFRecord 데이터셋을 저장하는 경로
3. `epochs` - epoch 횟수

아래 명령어로 **<font color='red'>SageMaker 관련 API 호출 없이</font>** 로컬 노트북 인스턴스 환경에서 1 epoch만 학습해 봅니다. 참고로, MacBook Pro(15-inch, 2018) 2.6GHz Core i7 16GB 사양에서 2분 20초~2분 40초 소요됩니다.

In [16]:
%%time
!mkdir -p logs
!python training_script/cifar10_keras_tf2.py --model_dir ./logs \
                                         --train data/train \
                                         --validation data/validation \
                                         --eval data/eval \
                                         --epochs 1
!rm -rf logs

Train for 312 steps, validate for 78 steps
312/312 [==============================] - 10s 32ms/step - loss: 1.8502 - accuracy: 0.3190 - val_loss: 1.7961 - val_accuracy: 0.3648
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./logs/1/assets
INFO:tensorflow:Assets written to: ./logs/1/assets
CPU times: user 327 ms, sys: 115 ms, total: 442 ms
Wall time: 20.1 s


**<font color='blue'>본 스크립트는 SageMaker상의 notebook에서 구동하고 있지만, 여러분의 로컬 컴퓨터에서도 python과 jupyter notebook이 정상적으로 인스톨되어 있다면 동일하게 수행 가능합니다.</font>**

# (2) Use TensorFlow Script Mode

TensorFlow 버전 1.11 이상에서 Amazon SageMaker Python SDK는 **스크립트 모드(Script mode)**를 지원합니다. 스크립트 모드는 종래 레거시 모드(Legacy mode) 대비 아래 장점들이 있습니다.

* 스크립트 모드의 학습 스크립트는 일반적으로 TensorFlow 용으로 작성하는 학습 스크립트와 더 유사하므로 TensorFlow 학습 스크립트를 최소한의 변경으로 실행할 수 있습니다. 따라서, 기존 레거시 모드보다 TensorFlow 학습 스크립트를 수정하는 것이 더 쉽습니다. 
    - 레거시 모드는 Tensorflow Estimator API를 기반으로 한 아래의 함수들을 반드시 포함해야 합니다.
        - 아래 함수들에서 하나의 함수를 만드시 포함해야 합니다.
            - `model_fn`: 학습할 모델을 정의합니다,
            - `keras_model_fn`: 학습할 tf.keras 모델을 정의합니다.
            - `estimator_fn`: 학습할 tf.estimator.Estimator를 정의합니다.
        - `train_input_fn`: 학습 데이터 로딩과 전처리를 수행합니다. 
        - `eval_input_fn`: 검증 데이터의 로딩과 전처리를 수행합니다.
        - (Optional) `serving_input_fn`: 예측(prediction) 중에 모델에 전달할 feautre를 정의합니다. 이 함수는 학습시에만 사용되지만, SageMaker 엔드포인트에서 모델을 배포할 때 필요합니다.
    - `if __name__ == “__main__”:` 블록을 정의할 수 없어 디버깅이 쉽지 않습니다.
    
* 스크립트 모드는 Python 2.7-와 Python 3.6-을 지원합니다.


TensorFlow 스크립트 모드에서 학습 스크립트를 작성하는 방법 및 Tensorflow 스크립트 모드의 estimator와 model 사용법에 대한 자세한 내용은
https://sagemaker.readthedocs.io/en/stable/using_tf.html 을 참조하세요.

### Preparing your script for training in SageMaker

SageMaker 스크립트 모드의 학습 스크립트는 SageMaker 외부에서 실행할 수 있는 학습 스크립트와 매우 유사합니다.
SageMaker는 하나의 인자값(argument), model_dir와 로그 및 모델 아티팩트(model artifacts)에 사용되는 S3 경로로 학습 스크립트를 실행합니다.

SageMaker 학습 인스턴스에서는 학습의 컨테이너에 S3에 저장된 데이터를 다운로드하여 학습에 활용합니다. 그 때, S3 버킷의 데이터 경로와 컨테이너의 데이터 경로를 컨테이너 환경 변수를 통해 연결합니다.

여러분은 다양한 환경 변수를 통해 학습 환경에 대한 유용한 속성들(properties)에 액세스할 수 있습니다.
이 스크립트의 경우 `Train, Validation, Eval`이라는 3 개의 데이터 채널을 스크립트로 보냅니다.


**아래의 단계별로 아래의 작업을 원본 소스에서 수정소스로 변경 됩니다. 
두 개의 파일을 비교하여 어떻게 변경이 되는지를 화인하시기 바랍니다.**
- 원본 소스: `training_script/cifar10_keras_tf2.py`
- 수정 소스: `training_script/cifar10_keras_sm_tf2.py`


----
### TODO 1.
`cifar10_keras_sm_tf2.py`파일에서 SageMaker API 환경 변수 SM_CHANNEL_TRAIN, SM_CHANNEL_VALIDATION, SM_CHANNEL_EVAL에서 디폴트 값을 가져오기 위해 train, validation, eval 인수를 수정해 주세요. 

`cifar10_keras_sm_tf2.py`의 `if __name__ == '__main__':` 블록 내에 아래 인자값을 수정해 주세요.

```python
parser.add_argument(
        '--train',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_TRAIN'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
parser.add_argument(
        '--validation',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_VALIDATION'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
parser.add_argument(
        '--eval',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_EVAL'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
```


환경 변수에 따른 S3 경로와 컨테이너 경로는 아래 표와 같습니다.

|  S3 경로  |  환경 변수  |  컨테이너 경로  |
| :---- | :---- | :----| 
|  s3://bucket_name/prefix/train  |  `SM_CHANNEL_TRAIN`  | `/opt/ml/input/data/train`  |
|  s3://bucket_name/prefix/validation  |  `SM_CHANNEL_VALIDATION`  | `/opt/ml/input/data/validation`  |
|  s3://bucket_name/prefix/eval  |  `SM_CHANNEL_EVAL`  | `/opt/ml/input/data/eval`  |
|  s3://bucket_name/prefix/model.tar.gz  |  `SM_MODEL_DIR`  |  `/opt/ml/model`  |
|  s3://bucket_name/prefix/output.tar.gz  |  `SM_OUTPUT_DATA_DIR`  |  `/opt/ml/output/data`  |

얘를 들어, `/opt/ml/input/data/train`은 학습 데이터가 다운로드되는 컨테이너 내부의 디렉토리입니다.

자세한 내용은 아래의 SageMaker Python SDK 문서를 확인하시기 바랍니다.<br>
(https://sagemaker.readthedocs.io/en/stable/using_tf.html#preparing-a-script-mode-training-script)


SageMaker는 train, validation, eval 경로들을 직접 인자로 보내지 않고, 대신 스크립트에서 환경 변수를 사용하여 해당 인자를 필요하지 않은 것으로 표시합니다.

SageMaker는 유용한 환경 변수를 여러분이 작성한 학습 스크립트로 보냅니다. 예시들은 아래와 같습니다.
* `SM_MODEL_DIR`: 학습 작업이 모델 아티팩트(model artifacts)를 저장할 수 있는 로컬 경로를 나타내는 문자열입니다. 학습 완료 후, 해당 경로 내 모델 아티팩트는 모델 호스팅을 위해 S3에 업로드됩니다. 이는 S3 위치인 학습 스크립트에 전달 된 model_dir 인수와 다르다는 점을 주의해 주세요. SM_MODEL_DIR은 항상 `/opt/ml/model`로 설정됩니다.
* `SM_NUM_GPUS`: 호스트(Host)에서 사용 가능한 GPU 수를 나타내는 정수(integer)입니다.
* `SM_OUTPUT_DATA_DIR`: 출력 아티팩트를 저장할 디렉토리의 경로를 나타내는 문자열입니다. 출력 아티팩트에는 체크포인트, 그래프 및 다른 저장용 파일들이 포함될 수 있지만 모델 아티팩트는 포함되지 않습니다. 이 출력 아티팩트들은 압축되어 모델 아티팩트와 동일한 접두사가 있는 S3 버킷으로 S3에 업로드됩니다.

이 샘플 코드는 네트워크 지연을 줄이기 위해 모델의 체크포인트(checkpoints)를 로컬 환경에 저장합니다. 이들은 학습 종료 후 S3에 업로드할 수 있습니다.

----
### TODO 2.

`cifar10_keras_sm_tf2.py`의 `if __name__ == '__main__':` 블록 내에 아래 인자값을 추가해 주세요.

```python
parser.add_argument(
        '--model_output_dir',
        type=str,
        default=os.environ.get('SM_MODEL_DIR'))
```

----
### TODO 3.
`ModelCheckpoint` 함수의 저장 경로를 새 경로로 아래와 같이 수정해 주세요.

From:
```python
callbacks.append(ModelCheckpoint(args.model_dir + '/checkpoint-{epoch}.h5'))
```
To:
```python
callbacks.append(ModelCheckpoint(args.model_output_dir + '/checkpoint-{epoch}.h5'))
```

----
### TODO 4.
`save_model` 함수의 인자값을 아래와 같이 수정해 주세요.

From:  
```python
return save_model(model, args.model_dir)
```
To:  
```python
return save_model(model, args.model_output_dir)
```



### Test your script locally (just like on your laptop)

테스트를 위해 위와 동일한 명령(command)으로 새 스크립트를 실행하고, 예상대로 실행되는지 확인합니다. <br>
SageMaker TensorFlow API 호출 시에 환경 변수들은 자동으로 넘겨기지만, 로컬 주피터 노트북에서 테스트 시에는 수동으로 환경 변수들을 지정해야 합니다. (아래 예제 코드를 참조해 주세요.)

```python
%env SM_MODEL_DIR=./logs
```

In [17]:
%%time
!mkdir -p logs   

# Number of GPUs on this machine
%env SM_NUM_GPUS=1
# Where to save the model
%env SM_MODEL_DIR=./logs
# Where the training data is
%env SM_CHANNEL_TRAIN=data/train
# Where the validation data is
%env SM_CHANNEL_VALIDATION=data/validation
# Where the evaluation data is
%env SM_CHANNEL_EVAL=data/eval

!python training_script/cifar10_keras_sm_tf2.py --model_dir ./logs --epochs 1
!rm -rf logs

env: SM_NUM_GPUS=1
env: SM_MODEL_DIR=./logs
env: SM_CHANNEL_TRAIN=data/train
env: SM_CHANNEL_VALIDATION=data/validation
env: SM_CHANNEL_EVAL=data/eval
Train for 312 steps, validate for 78 steps
312/312 [==============================] - 10s 32ms/step - loss: 1.8305 - accuracy: 0.3282 - val_loss: 1.6066 - val_accuracy: 0.3941
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./logs/1/assets
INFO:tensorflow:Assets written to: ./logs/1/assets
CPU times: user 350 ms, sys: 99.7 ms, total: 450 ms
Wall time: 20.2 s


# (3) Use SageMaker local for local testing

본격적으로 학습을 시작하기 전에 로컬 모드를 사용하여 디버깅을 먼저 수행합니다. 로컬 모드는 학습 인스턴스를 생성하는 과정이 없이 로컬 인스턴스로 컨테이너를 가져온 후 곧바로 학습을 수행하기 때문에 코드를 보다 신속히 검증할 수 있습니다.

Amazon SageMaker Python SDK의 로컬 모드는 TensorFlow 또는 MXNet estimator서 단일 인자값을 변경하여 CPU (단일 및 다중 인스턴스) 및 GPU (단일 인스턴스) SageMaker 학습 작업을 에뮬레이션(enumlate)할 수 있습니다. 이를 위해 Docker compose와 NVIDIA Docker를 사용합니다.
학습 작업을 시작하기 위해 `estimator.fit() ` 호출 시, Amazon ECS에서 Amazon SageMaker TensorFlow 컨테이너를 로컬 노트북 인스턴스로 다운로드합니다.

로컬 모드의 학습을 통해 여러분의 코드가 현재 사용 중인 하드웨어를 적절히 활용하고 있는지 확인하기 위한 GPU 점유와 같은 지표(metric)를 쉽게 모니터링할 수 있습니다.

In [18]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

`sagemaker.tensorflow` 클래스를 사용하여 SageMaker Python SDK의 Tensorflow Estimator 인스턴스를 생성합니다.
인자값으로 하이퍼파라메터와 다양한 설정들을 변경할 수 있습니다.

자세한 내용은 [documentation](https://sagemaker.readthedocs.io/en/stable/using_tf.html#training-with-tensorflow-estimator)을 확인하시기 바랍니다.

In [19]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1, 
                       train_instance_type='local')

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


학습을 수행할 3개의 채널과 데이터의 경로를 지정합니다. **로컬 모드로 수행하기 때문에 S3 경로 대신 노트북 인스턴스의 경로를 지정하시면 됩니다.**

In [20]:
%%time
estimator.fit({'train': 'file://data/train',
               'validation': 'file://data/validation',
               'eval': 'file://data/eval'})

Creating tmporecmmyy_algo-1-c1dnx_1 ... 
Attaching to tmporecmmyy_algo-1-c1dnx_12mdone
algo-1-c1dnx_1  | 2021-01-27 03:49:22,882 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-c1dnx_1  | 2021-01-27 03:49:22,889 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-c1dnx_1  | 2021-01-27 03:49:23,112 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-c1dnx_1  | 2021-01-27 03:49:23,131 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-c1dnx_1  | 2021-01-27 03:49:23,148 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-c1dnx_1  | 2021-01-27 03:49:23,160 sagemaker-containers INFO     Invoking user script
algo-1-c1dnx_1  | 
algo-1-c1dnx_1  | Training Env:
algo-1-c1dnx_1  | 
algo-1-c1dnx_1  | {
algo-1-c1dnx_1  |     "additional_framework_parameters": {},
algo-1-c1dnx_1  |     "channel_input_dirs": {
algo-1-c1dnx_1  

Estimator가 처음 실행될 때 Amazon ECR 리포지토리(repository)에서 컨테이너 이미지를 다운로드해야 하지만 학습을 즉시 시작할 수 있습니다. 즉, 별도의 학습 클러스터가 프로비저닝 될 때까지 기다릴 필요가 없습니다. 또한 반복 및 테스트시 필요할 수 있는 후속 실행에서 MXNet 또는 TensorFlow 스크립트에 대한 수정 사항이 즉시 실행되기 시작합니다.

# (4) Using SageMaker for faster training time

이번에는 로컬 모드를 사용하지 않고 SageMaker 학습에 GPU 학습 인스턴스를 생성하여 학습 시간을 단축해 봅니다.<br>
로컬 모드와 다른 점들은 (1) `train_instance_type`이 로컬 모드의 ‘local’ 대신 여러분이 원하는 특정 인스턴스 유형으로 설정해야 하고, (2) 학습 데이터를 Amazon S3에 업로드 후 학습 경로를 S3 경로로 설정해야 합니다. 

SageMaker SDK는 S3 업로드를 위한 간단한 함수(`Session.upload_data()`)를 제공합니다. 이 함수를 통해 리턴되는 값은 데이터가 저장된 S3 경로입니다.
좀 더 자세한 설정이 필요하다면 SageMaker SDK 대신 boto3를 사용하시면 됩니다.

*[Note]: 고성능 워크로드를 위해 Amazon EFS와 Amazon FSx for Lustre도 지원하고 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/*

In [21]:
dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-ap-northeast-2-057716757052/data/DEMO-cifar10'

S3에 데이터 업로드를 완료했다면, Estimator를 새로 생성합니다. <br>
아래 코드를 그대로 복사 후에 `train_instance_type='local'`을 `train_instance_type='ml.p2.xlarge'`로 수정하고
`hyperparameters={'epochs': 1}`를 `hyperparameters={'epochs': 5}`로 수정합니다.

```python
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 1},
                       train_instance_count=1, 
                       train_instance_type='local')
```

*[Note] 
2019년 8월부터 SageMaker에서도 학습 인스턴스에 EC2 spot instance를 사용하여 비용을 크게 절감할 수 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/ko/blogs/korea/managed-spot-training-save-up-to-90-on-your-amazon-sagemaker-training-jobs/*

만약 Managed Spot Instance로 학습하려면 다음 코드를 Estimator의 train_instance_type의 다음 행에 추가해 주세요.
```python
train_max_run = 3600,
train_use_spot_instances = 'True',
train_max_wait = 3600,
```

In [22]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 2},
                       train_instance_count=1, 
                       train_instance_type='ml.p3.8xlarge')

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


학습을 수행합니다. 이번에는 각각의 채널(`train, validation, eval`)에 S3의 데이터 저장 위치를 지정합니다.<br>
학습 완료 후 Billable seconds도 확인해 보세요. Billable seconds는 실제로 학습 수행 시 과금되는 시간입니다.
```
Billable seconds: <time>
```

참고로, `ml.p2.xlarge` 인스턴스로 5 epoch 학습 시 전체 6분-7분이 소요되고, 실제 학습에 소요되는 시간은 3분-4분이 소요됩니다.

In [23]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2021-01-27 03:50:46 Starting - Starting the training job...
2021-01-27 03:51:10 Starting - Launching requested ML instancesProfilerReport-1611719446: InProgress
......
2021-01-27 03:52:11 Starting - Preparing the instances for training......
2021-01-27 03:53:12 Downloading - Downloading input data...
2021-01-27 03:53:32 Training - Downloading the training image...
2021-01-27 03:54:13 Training - Training image download completed. Training in progress.2021-01-27 03:54:03,618 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-01-27 03:54:04,131 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
   

In [24]:
%store dataset_location

Stored 'dataset_location' (str)
